<a href="https://colab.research.google.com/github/FrantisekBrandejs/BAK-RQA/blob/main/BAK_RQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Využití RQA v kartografii** | bakalářská práce | František BRANDEJS, 2026

Prohlídnutí dat

In [ ]:
import pandas as pd
data = pd.read_csv('drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv', sep = '\t', nrows = 10) #načtení jen prvních 10 řádků
print(data)
print(data.columns.to_list()) #výpis sloupců
#!wc -l 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' #počet řádků
#!cut -f555 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' | sort -u #unikátní hodnoty ve specifickém sloupci
#!cut -f19 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' | sort -u #co za hodnoty ma eye movement type
!cut -f18 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' | sort -u

Chunk zpracování a redukce dat

In [ ]:
import pandas as pd
import re

file_path = 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv'
out_path = 'drive/MyDrive/BAK-RQA/reduced_data.tsv'

header = pd.read_csv(file_path, sep='\t', nrows=0)
all_columns = header.columns.tolist()

#Základní sloupce
base_cols = [
    'Participant name',
    'Recording timestamp',
    'Zkusenosti',
    'Eye movement type',
    'Gaze point X',
    'Gaze point Y',
    'Presented Stimulus name'
    ]

# všechny AOI sloupce (Ty co začínají AOI hit [)
aoi_cols = [c for c in all_columns if 'AOI hit [' in c]

#všechny sloupce, co se budou načítat
load_cols = base_cols + aoi_cols

#HLAVNÍ filtrace (chunk)
chunk_size = 100000 #kolik řádků se bude brát najednou
count = 0

print("---Začátek filtrace---")

reader = pd.read_csv(file_path, sep='\t', usecols=load_cols, chunksize=chunk_size)

for i, chunk in enumerate(reader):

    #filtrace STIMULŮ (začínající na R)
    filter_stimulus_R = chunk['Presented Stimulus name'].str.startswith('R').fillna(False)

    #filtrace dle EYE MOVEMENT TYPE
    filter_eye_movement_type = chunk['Eye movement type'] == 'Fixation'

    #filtrace AOI (Mapa vs Legenda)
    filter_legend = pd.Series(False, index=chunk.index)
    filter_all = pd.Series(False, index=chunk.index)
    filter_others = pd.Series(False, index=chunk.index)

    for col in aoi_cols:

      stim_in_col = col.split('[')[1].split(' -')[0]

      is_hit = (chunk[col] == 1) & (chunk['Presented Stimulus name'] == stim_in_col)

        if col.endswith('All]'):
            filter_all |= is_hit
        elif any(f' - 0{n}]' in col for n in range(1, 10)): # 01 až 09
            filter_legend |= is_hit
        else:
            filter_others |= is_hit

    AOI_pure_map = filter_all & ~filter_others & ~filter_legend
    final_filter = filter_stimulus_R & filter_eye_movement_type & (AOI_pure_map | filter_legend)

    reduced_chunk = chunk[final_filter].copy()

    reduced_chunk['AOI_Category'] = 'Map'
    reduced_chunk.loc[mask_legend, 'AOI_Category'] = 'Legend'

    if i == 0:
        reduced_chunk[base_cols + ['AOI_Category']].to_csv(out_path, sep='\t', index=False, mode='w')
    else:
        reduced_chunk[base_cols + ['AOI_Category']].to_csv(out_path, sep='\t', index=False, mode='a', header=False)

    count += len(chunk)
    if (i + 1) % 5 == 0:
        print(f"Zpracováno {count:,} řádků...")

print(f"Data uložena v: {out_path}")


In [ ]:
file_path = 'drive/MyDrive/BAK-RQA/reduced_data.tsv'

import pandas as pd
data = pd.read_csv(file_path, sep = '\t', nrows = 1) #načtení jen prvních 10 řádků
print(data)
print(data.columns.to_list()) #výpis sloupců
!wc -l 'drive/MyDrive/BAK-RQA/reduced_data.tsv' #počet řádků
!cut -f6 'drive/MyDrive/BAK-RQA/reduced_data.tsv' | sort -u # kontrola jestli redukovaná data jsou jen fixace